<a href="https://colab.research.google.com/github/Satwikram/Advanced-CNN-Architecture-Implementations/blob/master/Rice_Diseases_prediction_using_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"satwikram","key":"06c98063bdd0e68efaf2312bfe2c72df"}'}

In [3]:
! mkdir ~/.kaggle


In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download -d minhhuy2810/rice-diseases-image-dataset

100% 12.0G/12.0G [03:14<00:00, 42.6MB/s]
100% 12.0G/12.0G [03:14<00:00, 66.3MB/s]


In [ ]:
! unzip /content/rice-diseases-image-dataset.zip

In [18]:
# Importing Librarires
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os

In [9]:
# Setting image size
img_size = (224, 224, 3)

In [10]:
train_set = '/content/RiceDiseaseDataset/train'
test_set = '/content/RiceDiseaseDataset/validation'

In [11]:
# Importing VGG from Keras Applications
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [12]:
vgg = VGG16(input_shape = img_size, weights = 'imagenet', include_top = False)

58892288/58889256 [==============================] - 0s 0us/step


In [13]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [14]:
# useful for getting number of classes
from glob import glob
folders = glob('data/train/*')
print(len(folders))

0


In [15]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten

In [16]:
X = Flatten()(vgg.output)
prediction = Dense(4, activation = 'softmax')(X)
model = Model(inputs = vgg.input, outputs = prediction)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [20]:
sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [21]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [22]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [23]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [24]:
train_generator = train_datagen.flow_from_directory(
        train_set,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 1600 images belonging to 4 classes.


In [25]:
validation_generator = test_datagen.flow_from_directory(
        test_set,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 492 images belonging to 4 classes.


In [26]:
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [27]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

In [28]:
checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=1, save_best_only=True)



In [29]:
callbacks = [checkpoint, lr_reducer]

In [30]:
start = datetime.now()

In [31]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [32]:
model.fit_generator(
        train_generator,
        epochs= 10,
        validation_data=validation_generator,
        callbacks = callbacks)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
50/50 [==============================] - ETA: 0s - loss: 62.9986 - accuracy: 0.2569 
Epoch 00001: val_loss improved from inf to 60.55867, saving model to mymodel.h5
50/50 [==============================] - 1223s 24s/step - loss: 62.9986 - accuracy: 0.2569 - val_loss: 60.5587 - val_accuracy: 0.3110 - lr: 0.0100
Epoch 2/10
50/50 [==============================] - ETA: 0s - loss: 55.9825 - accuracy: 0.2756 
Epoch 00002: val_loss improved from 60.55867 to 39.33567, saving model to mymodel.h5
50/50 [==============================] - 1219s 24s/step - loss: 55.9825 - accuracy: 0.2756 - val_loss: 39.3357 - val_accuracy: 0.2500 - lr: 0.0100
Epoch 3/10
50/50 [==============================] - ETA: 0s - loss: 34.6881 - accuracy: 0.2788 
Epoch 00003: val_loss did not improve from 39.33567
50/50 [==============================] - 1221s 24s/step - loss: 34.6881 - accuracy: 0.2788 - val_loss: 43.8401 - val_accuracy

KeyboardInterrupt: ignored